# EDA

In [6]:
import pandas as pd
import numpy as np
import sweetviz
from sklearn.model_selection import train_test_split

# titanic data

In [3]:
path = '../../data/titanic.csv'
df = pd.read_csv(path)
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# Sweetviz

In [4]:
# analyze
eda_report = sweetviz.analyze([df, 'Data'], target_feat='Survived')

Done! Use 'show' commands to display/save.   |██████████| [100%]   00:00 -> (00:00 left)


In [5]:
# show html
eda_report.show_html("eda_report1.html")

Report eda_report1.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


## Train/test split

In [7]:
train, test = train_test_split(df, test_size = 0.2, random_state =42)

In [8]:
## compare train, test using sweetviz
compare_report = sweetviz.compare([train, 'Train'], [test, 'Test'], 'Survived')
compare_report.show_html("compare_report.html")

Done! Use 'show' commands to display/save.   |██████████| [100%]   00:00 -> (00:00 left)


Report compare_report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [11]:
# comparing sub-populations
compare_sex = sweetviz.compare_intra(df, df["Sex"]== "male", ["Male", "Female"], 'Survived')
compare_sex.show_html("compare_sex.html")

Done! Use 'show' commands to display/save.   |██████████| [100%]   00:00 -> (00:00 left)


Report compare_sex.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.
